# Simulação

In [1]:
"""Imports"""

import numpy as np
import pandas as pd
import codigo.simulacao as sim
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime

## Processando o DataSet

Começamos importando nosso dataset como um `DataFrame`.

In [2]:
df = pd.read_csv('dados/Production.csv')
df = df.drop(['EmergencyStop'], axis=1)

df_n_linhas = df.shape[0]
df_n_colunas = df.shape[1]
df_colunas = df.columns

df

,Timestamp,CupState,CurrentWeight,FanOn,HeatCoverClosed,HeaterOn,HeatingTimePercent,PotAvailable,Produce,ScaleCupDown,ScaleCupUp,SwitchToPot,TrackSwitchAtCup,TrackSwitchAtPot,TurnScaleCup
0,1516355920018,1,0.092367,0,1,0,0,1,0,0,1,0,1,0,0
1,1516355920120,1,0.153945,0,1,0,0,1,0,0,1,0,1,0,0
2,1516355920229,1,0.092367,0,1,0,0,1,0,0,1,0,1,0,0
3,1516355920339,1,0.123156,0,1,0,0,1,0,0,1,0,1,0,0
4,1516355920450,1,0.123156,0,1,0,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,1516356931913,1,0.554201,1,1,0,0,1,0,0,1,0,1,0,0
7724,1516356932022,1,0.584990,1,1,0,0,1,0,0,1,0,1,0,0
7725,1516356932131,1,0.584990,1,1,0,0,1,0,0,1,0,1,0,0
7726,1516356932241,1,0.584990,1,1,0,0,1,0,0,1,0,1,0,0


Passamos nossos temppos para um formato mais amigável

In [3]:
for index in range(df_n_linhas):
    unix_ts = df.at[index, 'Timestamp']
    df.at[index, 'Timestamp'] = datetime.fromtimestamp(unix_ts/1000)
df

,Timestamp,CupState,CurrentWeight,FanOn,HeatCoverClosed,HeaterOn,HeatingTimePercent,PotAvailable,Produce,ScaleCupDown,ScaleCupUp,SwitchToPot,TrackSwitchAtCup,TrackSwitchAtPot,TurnScaleCup
0,2018-01-19 06:58:40.018000,1,0.092367,0,1,0,0,1,0,0,1,0,1,0,0
1,2018-01-19 06:58:40.120000,1,0.153945,0,1,0,0,1,0,0,1,0,1,0,0
2,2018-01-19 06:58:40.229000,1,0.092367,0,1,0,0,1,0,0,1,0,1,0,0
3,2018-01-19 06:58:40.339000,1,0.123156,0,1,0,0,1,0,0,1,0,1,0,0
4,2018-01-19 06:58:40.450000,1,0.123156,0,1,0,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,2018-01-19 07:15:31.913000,1,0.554201,1,1,0,0,1,0,0,1,0,1,0,0
7724,2018-01-19 07:15:32.022000,1,0.584990,1,1,0,0,1,0,0,1,0,1,0,0
7725,2018-01-19 07:15:32.131000,1,0.584990,1,1,0,0,1,0,0,1,0,1,0,0
7726,2018-01-19 07:15:32.241000,1,0.584990,1,1,0,0,1,0,0,1,0,1,0,0


Iniciando os preparativos para nossa colea de tempos

In [4]:

df_aux = df.iloc[:7596]
dfaux_n_linhas = df_aux.shape[0]

periodos = [990, 1962, 2992, 3857, 4812, 5806, 6704, 7596]
tempos = {
    tempo: [] for tempo in [
        'Enchimento da panela', 'Ligação do aquecedor', 'Aquecimento da panela', 'Enchimento do copo', 'Despejo do copo'
    ]
}



Loop principal para medição de tempos

In [5]:
for periodo in periodos[:-1]:
    
    index = periodos.index(periodo)+1
    df = df_aux.iloc[periodo:periodos[index]]

    # 1 - Tempo de enchimento da panela
    tempo = df.query('CupState == 0')['Timestamp'].values
    tempo = tempo[-1] - tempo[0]
    tempos['Enchimento da panela'].append(float('{}.{}'.format(tempo.seconds, tempo.microseconds)))
    
    # 2 - Tempo entre momento que o processo iniciou e o aquecedor ligou
    tempo = df.query('HeaterOn == 0')['Timestamp'].values[0]
    tempo_aux = df.query('HeaterOn == 1')['Timestamp'].values[0]
    tempo = tempo_aux - tempo
    tempos['Ligação do aquecedor'].append(float('{}.{}'.format(tempo.seconds, tempo.microseconds)))
    
    # 3 - Tempo do momento que o aquecedor ligou, até começar a encher o copo de pipoca 
    tempo = df.query('HeaterOn == 1')['Timestamp'].values[0]
    tempo_aux = df.query('HeaterOn == 1').query('CurrentWeight > 1')['Timestamp'].values[0]
    tempo = tempo_aux - tempo
    tempos['Aquecimento da panela'].append(float('{}.{}'.format(tempo.seconds, tempo.microseconds)))
    
    # 4 - Tempo entre o momento que o copo começa a encher de pipoca até o momento que começa a despejar (CupState: 1 -> 2)
    tempo = df.query('HeaterOn == 1').query('CurrentWeight > 1')['Timestamp'].values[0]
    tempo_aux = df.query('CupState == 2')['Timestamp'].values[0]
    tempo = tempo_aux - tempo
    tempos['Enchimento do copo'].append(float('{}.{}'.format(tempo.seconds, tempo.microseconds)))
    
    # 5 - Tempo de despejo de pipoca
    tempo = df.query('CupState == 2')['Timestamp'].values[0]
    tempo_aux = df.iloc[-1]['Timestamp']
    tempo = tempo_aux - tempo
    tempos['Despejo do copo'].append(float('{}.{}'.format(tempo.seconds, tempo.microseconds)))

Assim, temos nossos tempos de execução como:

In [6]:
tempos = pd.DataFrame(tempos)
tempos

,Enchimento da panela,Ligação do aquecedor,Aquecimento da panela,Enchimento do copo,Despejo do copo
0,4.266,16.189,41.237,7.876,40.909
1,3.829,16.298,38.722,7.328,50.207
2,4.156,7.547,16.845,25.490,45.660
3,5.141,184.310,33.581,7.875,45.722
4,9.626,22.205,32.377,8.860,45.175
5,3.828,12.907,0.000,40.690,44.519
6,3.391,10.720,30.631,9.950,46.160


## Executado a simulação

Agora, vamos começar nossas simulações de fato, com nossa implementação importada como `sim`.

Para esse experimento, vamos verificar o desempenho do sistema em quatro cenários:
    
- Controle : Cenário original, sem alterações

  
- Com a panela sempre cheia : Cenário onde não esvaziamos a panela


- Com o aquecedor sempre ligado : Cenário onde não desligamos o aquecedor


- Ambas otimizações : Cenário onde aplicamos ambas práticas

Para facilitar nossos testes, escrevemos a seguinte rotina:

In [7]:
def executa_processo(tempos, limite_de_tempo, execucoes):
    
    #Definição de váriaveis containner
    
    figura, eixo = plt.subplots(1, 1)
    tempos_de_producao = {}

    
    #Ieração principal, para realizar a execução sobre cada cenário
    
    for estado_do_aquecedor in [False, True]:
        for estado_da_panela in [False, True]:
            
            if estado_do_aquecedor is False:
                categoria = "Controle" if estado_da_panela is False else "Panela sempre cheia"
            else:
                categoria = "Aquecedor sempre ligado" if estado_da_panela is False else "Ambas as otimizações"

            print(categoria)

            vetores = []

            # Executando o processo 'execucoes' vezes e guardando os resultados na nossa lista de vetores
            for _ in range(execucoes):
                processo = sim.Processo(
                    tempos,
                    aquecedor_sempre_ligado=estado_do_aquecedor,
                    panela_sempre_cheia=estado_da_panela,
                    verboso=False
                )

                vetores.append(processo.executa(limite_de_tempo) / 60)
                
            # Valor máximo de porções produzidas para o conjunto de execuções do cenário

            numero_maximo_de_porcoes = max([vetor.size for vetor in vetores])
            
            for v in range(len(vetores)):
                while vetores[v].size < numero_maximo_de_porcoes:
                    ultimo_valor = vetores[v][-1]
                    vetores[v] = np.append(vetores[v], ultimo_valor)

            # Formatações para o DataFrame
            
            for v in range(len(vetores)):
                vetores[v] = pd.DataFrame(np.concatenate(
                    (
                        np.full((numero_maximo_de_porcoes, 1), categoria),
                        np.full((numero_maximo_de_porcoes, 1), v),
                        np.arange(1, numero_maximo_de_porcoes + 1).reshape(-1, 1),
                        vetores[v].reshape(-1, 1)
                    ), axis=1
                ), columns=["Categoria", "Execução", "Porções produzidas", "Minutos decorridos"])

            tempos_de_producao[categoria] = pd.concat(vetores, axis=0)

    tempos_de_producao = pd.concat(tempos_de_producao.values(), axis=0)

    # tempos_de_producao.drop(
    #     tempos_de_producao.loc[tempos_de_producao.loc[:, "Minutos decorridos"] == "nan"].index, inplace=True
    # )

    for coluna in ["Execução", "Porções produzidas", "Minutos decorridos"]:
        tempos_de_producao.loc[:, coluna] = pd.to_numeric(tempos_de_producao.loc[:, coluna])

    sns.lineplot(data=tempos_de_producao, x="Porções produzidas", y="Minutos decorridos", hue="Categoria", ax=eixo)
    eixo.axvline(x=34, color="k", linestyle="--")

    return figura, tempos_de_producao

In [ ]:
executa_processo(tempos.to_numpy(), 3600, 1000)

Controle
